# Enviroment setting

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd
%cd /content/drive/Shared\ drives/rs/RSProject/recipe-personalization_org/
#%cd /content/drive/Shared\ drives/rs/RSProject/recipe-personalization/


/root
/content/drive/Shared drives/rs/RSProject/recipe-personalization_org


In [ ]:
! git clone https://github.com/majumderb/recipe-personalization.git
#%cd recipe-personalization/

Cloning into 'recipe-personalization'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 128 (delta 63), reused 92 (delta 50), pack-reused 0
Receiving objects: 100% (128/128), 77.37 KiB | 720.00 KiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
#PyTorch
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install torchvision
!pip install pytorch-pretrained-bert
!pip install pyrouge
import torch
#gpu enabling
use_cuda = False
#if use_cuda and torch.cuda.is_available():
 # print ("using GPU")
#    net.cuda()

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1MB 28kB/s 
     |████████████████████████████████| 8.8MB 57.5MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101
     |████████████████████████████████| 133kB 9.3MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-cp36-none-any.whl size=191613 sha256=9615869fb8dab65dd7620833b028f7270ead2fe76557c25fc79c87e6c098b130
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge


In [ ]:
%cd "/content/drive/My Drive/Shared_with_me/recipe-personalization_org/"

/content/drive/My Drive/Shared_with_me/recipe-personalization_org


In [ ]:

import pandas as pd
import recipe_gen
from recipe_gen import *
from recipe_gen import utils
from recipe_gen import analysis
from recipe_gen import pipeline
import recipe_gen.language
import recipe_gen.utils
import recipe_gen.pipeline
import recipe_gen.analysis
from recipe_gen.pipeline import *
from recipe_gen.analysis import *


100%|██████████| 458495/458495 [00:00<00:00, 530961.57B/s]
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


Prepering input files

In [ ]:
#modified_PP_recipes
df = pd.read_csv('/content/drive/My Drive/Shared_with_me/recipe-personalization_org/Data/recipe_reduced_ingr.csv', engine='c')
for col in ['name_tokens', 'ingredient_tokens', 'steps_tokens', 'techniques', 'ingredient_ids']:
    df[col] = df[col].agg(eval)
df.to_pickle('/content/drive/My Drive/Shared_with_me/recipe-personalization_org/DataTempEval/recipes.pkl')

In [ ]:
#Files creation: run only ONCE!
# No need to Run
import pandas as pd

interactions_train = pd.read_csv("Data/interactions_train.csv",nrows=1000)
interactions_train.to_pickle("DataTemp/interactions_train.pkl")
interactions_test = pd.read_csv("Data/interactions_test.csv")
interactions_test.to_pickle("DataTemp/interactions_test_new.pkl")
interactions_test = pd.read_csv("Data/interactions_test.csv")
interactions_test.to_pickle("DataTemp/interactions_test.pkl")
PP_u = pd.read_csv("Data/PP_users.csv")
PP_u.to_pickle("DataTemp/user_rep.pkl")
interactions_valid_new = pd.read_csv("Data/interactions_validation.csv")
interactions_valid_new.to_pickle("DataTemp/interactions_valid_new.pkl")
df = pd.read_csv('Data/PP_recipes.csv', engine='c')
for col in ['name_tokens', 'ingredient_tokens', 'steps_tokens', 'techniques', 'ingredient_ids']:
    df[col] = df[col].agg(eval)
df.to_pickle('DataTemp/recipes.pkl')

# Train

train on original pp_recipes

In [ ]:
!python3 -u -m recipe_gen.models.baseline.train --data-dir DataTemp/ --batch-size 32 --vocab-emb-size 300 --calorie-emb-size 5 --nhid 256 --nlayers 2 --lr 1e-3 --epochs 1 --annealing-rate 0.9 --save OUTPUT3K/ --ingr-emb --ingr-gru --exp-name baseline



In [ ]:
!python3 -u -m recipe_gen.models.user_nutri_pref.train --data-dir DataTemp50K/ --batch-size 38 --vocab-emb-size 300 --tech-emb-size 50 --calorie-emb-size 6 --nhid 256 --nlayers 2 --lr 1e-3 --epochs 7 --annealing-rate 0.9 --save OUTPUT50FNutri/ --ingr-emb --ingr-gru --exp-name usernutri
    



In [ ]:
!python3 -u -m recipe_gen.models.user_technique_pref.train --data-dir DataTemp/ --batch-size 32 --vocab-emb-size 300 --tech-emb-size 50 --calorie-emb-size 5 --nhid 256 --nlayers 2 --lr 1e-3 --epochs 1 --annealing-rate 0.9 --save OUTPUTT/ --ingr-emb --ingr-gru --exp-name tech_pref

# Test

In [ ]:
#this one:
#just set OUTPUT/model_baseline_e0.pt to the currect model file
!python3 -m recipe_gen.models.baseline.test --data-dir DataTemp/ --model-path OUTPUT12KBH/model_baseline_e4.pt --vocab-emb-size 300 --calorie-emb-size 5 --nhid 256 --nlayers 2 --ingr-emb --ingr-gru --ingr-emb-size 10 --save-dir OUTPUT12KBH/ --batch-size 32


In [ ]:
 !python3 -u -m recipe_gen.models.user_nutri_pref.test --data-dir DataModified/ --model-path OUTPUT50FNutri/model_usernutri_eX.pt --batch-size 38 --vocab-emb-size 300  --calorie-emb-size 6 --nhid 256 --nlayers 2 --ingr-emb --ingr-gru --ingr-emb-size 10 --save-dir OUTPUT50FNutri/ --batch-size 36 --ingr-gru

In [ ]:
!python -m recipe_gen.models.user_technique_pref.test --data-dir DataTempEval/ --model-path OUTPUTEvalTch/model_tech_pref_e2.pt --vocab-emb-size 300 --tech-emb-size 50 --calorie-emb-size 5 --nhid 256 --nlayers 2 --save-dir OUTPUTEvalTch/ --ingr-gru --ingr-emb --overwrite --batch-size 48


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Device: cuda:0
Using CUDA 0
0:00:00.062764 - Loaded 11,996 training interactions, 7,023 validation, 10 test (2.034 MB total memory)
0:00:00.322293 - Loaded items for 25,076 users, 6,437 maximum interactions/user (31.008 MB total memory)
0:00:06.609838 - Loaded 178,265 recipes (364.728 MB total memory)
0:00:06.715497 - Loaded map for 8,024 unique ingredients
0:00:06.756007 - Data loaded.
INGR EMBEDDING
0:00:00.802490 - Padded names to maximum 15 tokens
0:00:01.976910 - Padded steps to maximum 256 tokens
0:00:03.338434 - Clipped ingredients randomly between 3 and 5 incidents
0:00:06.203272 - Padded ingredients to maximum 20 tokens and 5 ingredients
0:00:07.476590 - Padded ingredient IDs to 

Review some output examples

In [ ]:
!cat "/content/drive/My Drive/Shared_with_me/recipe-personalization_org/OUTPUT50KBH/model_baseline_e2/test_i173538_u2/output.txt"


ORIGINAL NAME: `strawberry fudge |END|`
Calorie-Level: `Low-Calorie`
Techniques:
--boil
--combine
--pour
Ingredients:
--sugar
--strawberry gelatin
--salt
--milk
--half-and-half

Original Steps:
combine first 6 ingredients in a buttered 3 - quart heavy sauce pan . bring mixture to a boil over medium heat , stirring constantly . cook without stirring until candy thermometer reaches 230f . or until it makes a soft ball when tested in cold water . remove from heat . add margarine or butter and vanilla , do not stir ! . cool without stirring until outside of pan becomes lukewarm . stir in nuts and beat until candy loses its gloss . pour into 3 dozen paper bonbon cups . |END|

MODEL OUTPUT:
combine sugar , milk , and salt in a large saucepan and bring to boil , stirring constantly . boil , stirring occasionally , until mixture comes to a boil . stir in cream and vanilla . bring to a boil , then reduce heat and simmer for 10 minutes . remove from heat and stir in vanilla . pour into a grease

In [ ]:
!cat "/content/drive/My Drive/Shared_with_me/recipe-personalization_org/OUTPUTEvalTch/model_tech_pref_e2/test_i173538_u2/output.txt"


ORIGINAL NAME: `strawberry fudge |END|`
Calorie-Level: `Low-Calorie`
Techniques:
--c0
--s9
--f0
--n0
Ingredients:
--sugar
--strawberry gelatin
--salt
--milk
--half-and-half

Original Steps:
combine first 6 ingredients in a buttered 3 - quart heavy sauce pan . bring mixture to a boil over medium heat , stirring constantly . cook without stirring until candy thermometer reaches 230f . or until it makes a soft ball when tested in cold water . remove from heat . add margarine or butter and vanilla , do not stir ! . cool without stirring until outside of pan becomes lukewarm . stir in nuts and beat until candy loses its gloss . pour into 3 dozen paper bonbon cups . |END|

MODEL OUTPUT:
combine sugar and milk in a large bowl . add 1 / 2 cup sugar , 2 tbsp vanilla , 1 / 2 cup sugar , and 2 tablespoons water . beat until well combined , about 3 minutes . add vanilla and vanilla . beat until smooth . pour into a greased 13x9 pan . bake at 350f for 30 minutes . cool in pan on a wire rack . refr

In [ ]:
!cat "/content/drive/My Drive/Shared_with_me/recipe-personalization_org/OUTPUT50KBH/model_baseline_e2/test_i177847_u16/output.txt"


ORIGINAL NAME: `azumaya pot stickers |END|`
Calorie-Level: `Low-Calorie`
Techniques:
--combine
--simmer
--skillet
Ingredients:
--ground lean pork
--cabbage
--water chestnut
--scallion

Original Steps:
combine meat or tofu and cabbage , water chestnuts , green onions , ginger , soy sauce and salt in a bowl and mix well . place one teaspoon of filling in each wrap , fold and seal edges , turn pot sticker seam side up , then set down firmly on a flat surface to create a flat bottom . divide potstickers into 3 batches and cook each batch as follows . heat 2 teaspoons oil in a large nonstick skillet . brown bottoms of potstickers . add 1 cup of broth , cover skillet and let simmer 10 minutes . uncoverand cook until all liquid is absorbed . remove and serve with dipping sauces . |END|

MODEL OUTPUT:
in a small sauce pan , heat oil , add the garlic , and stir to cook . add the garlic and saut until the onions begin to soften . add the garlic and cook until the garlic is softened , about 5 mi

In [ ]:
!cat "/content/drive/My Drive/Shared_with_me/recipe-personalization_org/OUTPUTEvalTch/model_tech_pref_e2/test_i177847_u16/output.txt"


ORIGINAL NAME: `azumaya pot stickers |END|`
Calorie-Level: `Low-Calorie`
Techniques:
--c0
--s0
--f0
--n1
Ingredients:
--ground lean pork
--cabbage
--water chestnut
--scallion

Original Steps:
combine meat or tofu and cabbage , water chestnuts , green onions , ginger , soy sauce and salt in a bowl and mix well . place one teaspoon of filling in each wrap , fold and seal edges , turn pot sticker seam side up , then set down firmly on a flat surface to create a flat bottom . divide potstickers into 3 batches and cook each batch as follows . heat 2 teaspoons oil in a large nonstick skillet . brown bottoms of potstickers . add 1 cup of broth , cover skillet and let simmer 10 minutes . uncoverand cook until all liquid is absorbed . remove and serve with dipping sauces . |END|

MODEL OUTPUT:
place the chicken pieces in a large bowl , and stir in 1 tsp of water . add in the garlic , salt and pepper . add the remaining ingredients and stir well . add 1 tbsp of the butter to the pan . add the r

Review model stats

In [ ]:

df_stats = pd.read_pickle("ingredient_attention.pkl")
df_stats.head(4)

ingredient_attention.pkl  output.txt


,<s>,mix</w>,the</w>,first</w>,seven</w>,ingredients</w>,in</w>,a</w>,small</w>,bowl</w>,</s>,<s>,mix</w>,the</w>,dressing</w>,ingredients</w>,together</w>,and</w>,mix</w>,well</w>,</s>,<s>,pour</w>,mixture</w>,into</w>,the</w>,prepared</w>,pan</w>,</s>,<s>,bake</w>,at</w>,400</w>,degrees</w>,for</w>,20</w>,minutes</w>,</s>,<s>,serve</w>,...,if</w>,desired</w>,</s>,</R>,</s>,<s>,the</w>,chicken</w>,should</w>,be</w>,a</w>,little</w>,sticky</w>,</s>,<s>,serve</w>,over</w>,the</w>,chicken</w>,",</w>",lettuce</w>,",</w>",and</w>,salsa</w>,</s>,</R>,and</w>,a</w>,good</w>,football</w>,of</w>,black</w>,olives</w>,</s>,</R>,chicken</w>,",</w>",a</w>,few</w>,c
plain yogurt,0.120862,0.198283,0.111036,0.123241,0.221190,0.107316,0.079094,0.172799,0.175287,0.080974,0.093218,0.256074,0.240659,0.155391,0.107571,0.289098,0.169885,0.179096,0.169736,0.204865,0.137126,0.204240,0.252579,0.219682,0.214459,0.178143,0.102025,0.155745,0.239340,0.161714,0.251364,0.202284,0.131169,0.413893,0.284585,0.253028,0.314587,0.211146,0.240227,0.304192,...,0.231433,0.300028,0.156930,0.264176,0.135598,0.131380,0.141916,0.332129,0.250301,0.286450,0.271067,0.285866,0.228575,0.323058,0.399875,0.258651,0.218137,0.323722,0.078102,0.046374,0.242830,0.232908,0.177457,0.246950,0.202368,0.135656,0.175653,0.090308,0.310652,0.174416,0.180883,0.243075,0.204525,0.219685,0.071697,0.225099,0.103066,0.072877,0.220816,0.241750
chicken breast,0.225498,0.381452,0.658146,0.357025,0.450031,0.521616,0.343188,0.315920,0.264330,0.115044,0.121517,0.132483,0.284299,0.349143,0.317631,0.361316,0.326182,0.195224,0.233853,0.459366,0.262255,0.184400,0.297586,0.313062,0.393282,0.318588,0.290250,0.337804,0.252479,0.284587,0.408389,0.171587,0.396456,0.265560,0.377782,0.530341,0.292639,0.126161,0.445625,0.238009,...,0.403599,0.214760,0.337959,0.460321,0.523617,0.569417,0.411213,0.183859,0.304250,0.339226,0.159950,0.290735,0.276616,0.158738,0.284208,0.403723,0.395219,0.233113,0.285934,0.225375,0.376602,0.250687,0.410227,0.390615,0.286973,0.378810,0.263196,0.378327,0.285825,0.242957,0.241509,0.227641,0.322636,0.243732,0.431096,0.405931,0.496519,0.522232,0.243305,0.357334
white onion,0.602634,0.359718,0.223095,0.476557,0.221481,0.300188,0.545254,0.317220,0.160458,0.163533,0.067943,0.024011,0.048936,0.313986,0.467793,0.127717,0.059589,0.121411,0.113867,0.129190,0.195825,0.155137,0.069726,0.285806,0.162337,0.343537,0.424151,0.269920,0.303148,0.461980,0.077774,0.130569,0.103565,0.032949,0.036644,0.027407,0.045513,0.009326,0.034407,0.086568,...,0.070331,0.094355,0.158070,0.110748,0.155452,0.175475,0.333532,0.200676,0.103231,0.065915,0.064129,0.097510,0.079041,0.076799,0.087420,0.095940,0.151782,0.082256,0.528912,0.669710,0.257686,0.209737,0.109770,0.130463,0.121519,0.234291,0.268331,0.377868,0.122994,0.069535,0.081004,0.024437,0.151519,0.105631,0.150209,0.159187,0.238096,0.254572,0.321196,0.208404
scallion,0.051006,0.060547,0.007723,0.043176,0.107297,0.070880,0.032464,0.194061,0.399926,0.640449,0.717322,0.587431,0.426106,0.181480,0.107005,0.221869,0.444344,0.504269,0.482544,0.206579,0.404794,0.456223,0.380109,0.181450,0.229923,0.159732,0.183573,0.236532,0.205033,0.091719,0.262473,0.495560,0.368810,0.287597,0.300988,0.189225,0.347260,0.653368,0.279740,0.371230,...,0.294638,0.390857,0.347041,0.164755,0.185333,0.123727,0.113339,0.283336,0.342218,0.308409,0.504854,0.325888,0.415768,0.441405,0.228497,0.241685,0.234863,0.360908,0.107052,0.058542,0.122882,0.306668,0.302546,0.231972,0.389140,0.251242,0.292820,0.153496,0.280529,0.513091,0.496605,0.504847,0.321320,0.430952,0.346998,0.209783,0.162319,0.150318,0.214683,0.192512


# NutriScore calculation

In [ ]:
df = pd.read_csv('Data/PP_recipes.csv', engine='c')
for col in ['name_tokens', 'ingredient_tokens', 'steps_tokens', 'techniques', 'ingredient_ids']:
    df[col] = df[col].agg(eval)
df.to_pickle('DataTemp/recipes.pkl')

In [ ]:
df = pd.read_csv('DataModified/PP_recipes.csv', engine='c')
#RAW_recipes_modified.pkl

changing PP_Recipes data file

In [ ]:
#read two input files
#df.to_pickle("Data/RAW_recipes_modified.pkl")
df_recipes_raw_mod = pd.read_pickle("Data/RAW_recipes_modified.pkl")
df_pp_rec = pd.read_csv('Data/PP_recipes.csv', engine='c')
for col in ['name_tokens', 'ingredient_tokens', 'steps_tokens', 'techniques', 'ingredient_ids']:
    df_pp_rec[col] = df_pp_rec[col].agg(eval)

# selecting the relevant columns (id + nutriscore)

cols = ['id','nutriscore']
df_rec_raw_temp = df_recipes_raw_mod[cols]

#merge by key (id)
result = pd.merge(df_pp_rec,df_rec_raw_temp, on='id')
#drop old column (calorie level)
result = result.drop(['calorie_level'],axis=1)
#rename new column
result = result.rename(columns={"nutriscore": "calorie_level"})
#save new DF to file
result.to_csv('DataTemp/PP_recipes.csv')

NutriScore functions

In [ ]:

import numpy as np
import pandas as pd
from keras.utils import to_categorical


  def convert_dflist_to_list (feat):
    f_list = []
    temp_str = feat.split("[")[1].split("]")[0].split(",")
    for item in temp_str:
      C = int(item)
      f_list.append(C)
    return f_list



def PDV_to_grams(val, max_val):
  return (max_val * val / 100)
  
def nutri_score(val, scale):
  i = 0
  while (i < 10) :
    if val < scale[i]:
    #  print ("in if: i=",i,"val",val,"is smaller than scale:",scale[i])
      return (i)
    else:
      i +=1
  return (i)  
      

def calc_nutrition_grade(nutri_feat):
 # zeros = np.zeros(11)
  nutri_list = []
  temp_str = nutri_feat.split("[")[1].split("]")[0].split(",")
  for item in temp_str:
      C = float(item)
      nutri_list.append(C)
  
  #print (nutri_list)
  max_val_list = [2000, 65, 31, 2.3, 50, 20, 300]
  #1: cals 2: fats 3:sugar 4: sodium 5. protein 6: saturated fat 7: carbohydrates
  cals, tot_fats_dv, sugar_dv, sodium_dv, protein_dv, satur_fat_dv, carbons_dv = [nutri_list[i] for i in range (7)]
  max_cals, max_tot_fats, max_sugar, max_sodium, max_protein, max_satur_fat, max_carbons = [nutri_list[i] for i in range (7)]
  tot_fats = PDV_to_grams(tot_fats_dv, max_tot_fats)
  sugar = PDV_to_grams(sugar_dv, max_sugar)
  sodium = PDV_to_grams(sodium_dv, max_sodium)
  satur_fat = PDV_to_grams(satur_fat_dv, max_satur_fat)
  
  #positives: protein
  #negatives: sugar, calories, fatty acids, sodium
  #print ("cals",cals,"fats",satur_fat,"sugar",sugar,"sodium",sodium)
  calories_scale = np.array([335,670,1005,1340,1675,2010,2345,2680,3015,3350])
  sugars_scale = np.array([4.5,9,13.5,18,22.5,27,31,36,40,45])
  fats_scale = np.asarray([i for i in range (1,11)])
  sodium_scale = np.array([0.09,0.18,0.27,0.36,0.45,0.54,0.63,0.72,0.81,0.9])

  a = nutri_score(cals, calories_scale)
  b = nutri_score(sugar, sugars_scale)
  c = nutri_score(satur_fat, fats_scale)
  d = nutri_score(sodium, sodium_scale)

  final_score = [a,b,c,d]
  result = np.zeros((4,11))
  data = np.array(final_score)
  encoded = to_categorical(data,dtype='int32')#.reshape(1,44)
  result [:encoded.shape[0],:encoded.shape[1]] = encoded
  result = result.reshape(1,44).astype(int)
  return result
'''
#MAIN
df = pd.read_csv('Data/RAW_recipes.csv', engine='c')
samples  = df['nutrition']
decoded = []
for g in range (len(samples)):
   # print ("new sample")
    t = samples.iloc[g]
    f = calc_nutrition_grade(t)
   # decoded.append(f)
#print (decoded)
'''

'\n#MAIN\ndf = pd.read_csv(\'Data/RAW_recipes.csv\', engine=\'c\')\nsamples  = df[\'nutrition\']\ndecoded = []\nfor g in range (len(samples)):\n   # print ("new sample")\n    t = samples.iloc[g]\n    f = calc_nutrition_grade(t)\n   # decoded.append(f)\n#print (decoded)\n'

In [ ]:

samples  = df['nutrition']
temp = []
for g in range (len(samples)):
    t = samples.iloc[g]
    f = calc_nutrition_grade(t)
    temp.append(f)
#trying to modified original recipes file
temp_arr = np.asarray(temp)
temp_arr = temp_arr.reshape(len(temp_arr),44)
print (temp_arr.shape)
temp_l = temp_arr.tolist()
df_temp = pd.DataFrame({'nutri_encoded':temp_l})
df.reset_index(drop=True,inplace=True)
df_temp.reset_index(drop=True,inplace=True)
result = pd.concat([df,df_temp],axis=1)
result.head(10)
df_m = pd.merge(rec,result,how='inner',on='id')
rec_col = rec.columns
all_col = df_m.columns


In [ ]:
df_m = df_m.drop(columns=['name', 'minutes','techniques','contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'steps','description', 'ingredients', 'n_ingredients'])

In [ ]:
df_m = df_m.rename(columns={'nutri_encoded':'techniques'})

In [ ]:
#OK!
df_m.to_csv("Data/modified_PP_recipes.csv")

# NutriVecs

In [ ]:
users_items = users['items'] #slice only items columns. each row belongs to diffenet user
nutrivecs = [] #this will eventually keep whole nutrivecs to replace with techniques
for g in range (len(users_items)):
  #for each user in samples we need to calculated nutrivector (sum)
  item_list = users_items.iloc[g] #take a users' items: this is a string
  #print (len(u_items))
  #item_list = convert_dflist_to_list(u_items) #produce list object of items
  tech_l = [] #this is not really techniques, just a list to keep users' "techniques" - nutrivecs
  #nutri_vec = []
  for item in item_list: #for each item (rec) in user items list
  #print (item)
    rslt_df = rec[rec['i'] == item] #find "techniques" ohv belongs to this specific item=reciepe
    tech_l.append(rslt_df['techniques'].tolist())
  tech_arr = np.asarray(tech_l) #brings to array shape to fix dimensions
  tech_arr = tech_arr.reshape(len(tech_arr),44) #fix dimension
  nutri_vec = tech_arr.sum(axis=0).tolist()
  nutrivecs.append(nutri_vec)


In [ ]:
temp_arr = np.asarray(nutrivecs)
temp_arr = temp_arr.reshape(len(temp_arr),44)
print (temp_arr.shape)
temp_l = temp_arr.tolist()
df_nutri_temp = pd.DataFrame({'nutrivec':temp_l})
users.reset_index(drop=True,inplace=True)
df_nutri_temp.reset_index(drop=True,inplace=True)
result = pd.concat([users,df_nutri_temp],axis=1)
result.to_pickle('Data/modified_user_rep.pkl')
result = result.drop(columns=['techniques'])
result = result.rename(columns={'nutrivec':'techniques'})
result.to_pickle('DataTempCal/user_rep.pkl')

(25076, 44)


In [ ]:
'''
I have users items (recs) in u_items. so for each recipe in u_items, i ran over recipes table.
i extract the techs (encoded as ohv), receive final sparse tech_matrixs for all users' recipes.
than i sum them, and got one summerize vector of tech. I compare it to tech vector in users table 
to ensure we have the same values.
'''

item_list = convert_dflist_to_list(u_items)

tech_l = []
for item in item_list:
  #print (item)
  rslt_df = rec[rec['i'] == item]
  tech_l.append(rslt_df['techniques'].tolist())
tech_arr = np.asarray(tech_l)
tech_arr = tech_arr.reshape(len(tech_arr),58)
print (tech_arr.shape)

[122140, 77036, 156817, 76957, 68818, 155600, 15142, 107484, 141253, 161084, 38511, 97459, 71971, 139141, 71237, 47107, 155789, 5904, 150772, 128425, 137945, 138873, 132276, 39503, 95552, 4446, 90037, 130117, 46218, 138166, 68243, 78551, 111427, 144656, 134610, 87638, 102591, 0, 65028]
(39, 58)


# Create 50K interactions training file



In [ ]:
import pandas as pd

int_df = pd.read_csv("Data/interactions_train.csv",nrows=100)
int_df.head(1)

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367


In [ ]:
int_df.groupby('recipe_id')['user_id'].count()

recipe_id
100      1
164      1
191      1
278      1
350      1
        ..
9378     1
9430     1
13307    1
48296    1
51964    1
Name: user_id, Length: 98, dtype: int64

In [ ]:
df = pd.read_csv("Data/interactions_train.csv")
#picking users that interact with 30 to 50 recipes
filtered_df = df.groupby('user_id').filter(lambda s: s.recipe_id.count() > 30 and s.recipe_id.count() < 50)
filtered_df.to_pickle("DataTemp50K/interactions_train.pkl")

<bound method NDFrame.describe of         user_id  recipe_id        date  rating      u       i
13         2178       3704  2000-10-30     3.0   6065    3172
14         2178       4366  2000-11-04     5.0   6065   50924
19         2695        350  2001-01-19     1.0   9204   13543
54         2178       5478  2001-05-24     3.0   6065  135342
117        1634       8909  2001-07-02     3.0   7299   35059
...         ...        ...         ...     ...    ...     ...
698777   885024     102833  2018-11-16     5.0   9891   26175
698781   163077     243729  2018-11-17     5.0   3485   72440
698791  1580637     329676  2018-11-21     3.0  12999   91046
698835   536962     122554  2018-12-03     5.0   9244   60659
698894   117501     529600  2018-12-17     5.0  10368   18120

[48336 rows x 6 columns]>

In [ ]:
filtered_df.to_pickle("DataTemp50K/interactions_train.pkl")